In [40]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from math import sqrt
import random

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, NodesAndLinkedEdges, LabelSet, HoverTool, TapTool, BoxSelectTool, ResetTool, BoxZoomTool, ColorBar, BasicTicker, Div, Panel, Column, Tabs
from bokeh.plotting import figure, from_networkx, output_file
from bokeh.transform import linear_cmap, LinearColorMapper
from bokeh.colors import RGB

output_notebook()

Loading BokehJS ...

In [47]:
# GOT Edges sourced from https://melaniewalsh.github.io/Intro-Cultural-Analytics/00-Datasets/00-Datasets.html
df = pd.read_csv('got-edges.csv')
df['Weight_scaled'] = df.Weight / 100

# Making dummy labels
node_series = df.Source.apply(lambda x: random.choice(['Fizz', 'Buzz', 'Fizzbuzz']))
df['Node_label'] = node_series
df

,Source,Target,Weight,Weight_scaled,Node_label
0,Aemon,Grenn,5,0.05,Fizz
1,Aemon,Samwell,31,0.31,Fizzbuzz
2,Aerys,Jaime,18,0.18,Fizz
3,Aerys,Robert,6,0.06,Fizzbuzz
4,Aerys,Tyrion,5,0.05,Fizzbuzz
...,...,...,...,...,...
347,Walder,Petyr,6,0.06,Buzz
348,Walder,Roslin,6,0.06,Fizzbuzz
349,Walton,Jaime,10,0.10,Fizz
350,Ygritte,Qhorin,7,0.07,Buzz


In [48]:
G = nx.from_pandas_edgelist(df, source='Source', target='Target', edge_attr='Weight')

In [49]:
source2label = {source : label for source, label in zip(df.Source, df.Node_label)}
target2label = {target : label for target, label in zip(df.Target, df.Node_label)}
source2label.update(target2label)

In [51]:
# Title
title = 'GOT Network'
output_file(filename="GOT-Network.html", title=title)

# Set colors of Source and Target, assign weights
modularity_color, modularity_size, labels = {}, {}, {}

communities = nx.algorithms.community.greedy_modularity_communities(G)
for community_set in communities:
    for name in community_set:
        if name in set(df.Source):
            modularity_color[name] = '#2c284f'
            modularity_size[name] = 30
        else:
            modularity_color[name] = '#96a9a9'
            # modularity_size[name] = max(min(G.degree(name) * 5, 60), 10)
            modularity_size[name] = 30
        labels[name] = source2label.get(name, None)

nx.set_node_attributes(G, modularity_color, 'modularity_color')
nx.set_node_attributes(G, modularity_size, 'modularity_size')
nx.set_node_attributes(G, labels, 'labels')

# Create a plot — set dimensions, toolbar, and title
plot = figure(tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom', plot_width=1000, plot_height=600, title=title, x_range=Range1d(-12.1, 12.1), y_range=Range1d(-12.1, 12.1))
plot.add_tools(TapTool(), BoxSelectTool())

node_hover_tool = HoverTool(tooltips=[("Node label", "@labels")])
plot.add_tools(node_hover_tool, BoxZoomTool())

# Create a network graph object with spring layout
eq = 1 / sqrt(len(df)) * 5
network_graph = from_networkx(G, nx.spring_layout(G, k=eq, iterations=5, scale=12), seed=42, scale=12, center=(0, 0))
# Add color bar
m_rgb = (255 * plt.get_cmap("viridis_r")(range(256))).astype("int")
palette = [RGB(*tuple(rgb)).to_hex() for rgb in m_rgb]
color_mapper = LinearColorMapper(palette, low=5, high=9)
color_bar = ColorBar(color_mapper=color_mapper,
                     ticker=BasicTicker(desired_num_ticks=5),
                     location=(0, 0), title='Relationship weight',
                     major_label_text_font_size='17px',
                     title_text_font_size='19px',
                     title_text_font_style='bold', width=17, )
plot.add_layout(color_bar, 'right')

# Set node size and color 
network_graph.node_renderer.glyph = Circle(size='modularity_size', fill_color='modularity_color', line_width=0)

# Add edge color
network_graph.edge_renderer.glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.6)

# Edge weight
weight_map = dict(zip(zip(df.Source, df.Target), df.Weight_scaled))
weight_map.update(zip(zip(df.Target, df.Source), df.Weight_scaled))
data = network_graph.edge_renderer.data_source.data
data["line_width"] = [weight_map[edge] * 10 for edge in zip(data["start"], data["end"])]
# network_graph.edge_renderer.glyph.line_width = {'field': 'line_width'}
network_graph.edge_renderer.glyph.line_width = 2

# Interactivity
network_graph.node_renderer.selection_glyph = Circle(size='modularity_size', fill_color='#ffffff', line_color='modularity_color', line_width=5)
# network_graph.edge_renderer.selection_glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.8, line_width={'field': 'line_width'})
network_graph.edge_renderer.selection_glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.8, line_width=5)

network_graph.node_renderer.nonselection_glyph = Circle(size='modularity_size', fill_color='modularity_color', line_width=0, fill_alpha=0.8)
# network_graph.edge_renderer.nonselection_glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.1, line_width={'field': 'line_width'})
network_graph.edge_renderer.nonselection_glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.1, line_width=1)

# Hover
network_graph.node_renderer.hover_glyph = Circle(size='modularity_size', fill_color='#ffffff', line_color='modularity_color', line_width=5)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color={'field': 'Weight', 'transform': color_mapper}, line_alpha=0.8, line_width=5)

network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

# Main plot
plot.renderers.append(network_graph)
plot.axis.visible = False
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None

# Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white',  text_font_size='10px', background_fill_alpha=.9)
plot.renderers.append(labels)
nl = '\n'
disclaimer_text = "Network of interactions between GOT characters. Hover / click to highlight node connections."
disclaimer = Div(text=f'<font size="-1"><i><p style="color:black">{disclaimer_text.replace(nl, "<br>")}</p></i></font>', width=1500, height=30)
# save(Column(plot, disclaimer))
show(Column(plot, disclaimer))